In [1]:
import pandas as pd
import requests
import json

In [2]:
url = "https://api-publica.datajud.cnj.jus.br/api_publica_tjsp/_search"
api_key = "APIKey cDZHYzlZa0JadVREZDJCendQbXY6SkJlTzNjLV9TRENyQk1RdnFKZGRQdw==" # Chave pública
numero_processo = 15000156520218260451 # Exemplos: 15000156520218260451, 15072172520238260451

payload = json.dumps({
  "query": {
    "match": {
      "numeroProcesso": numero_processo
    }
  }
})

headers = {
  'Authorization': api_key,
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)  # <Response [200]>
dados_dict = response.json() # <class 'dict'>
print(dados_dict)

{'took': 29, 'timed_out': False, '_shards': {'total': 20, 'successful': 20, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1, 'relation': 'eq'}, 'max_score': 14.8171215, 'hits': [{'_index': 'api_publica_tjsp', '_type': '_doc', '_id': 'TJSP_279_G1_10067_15000156520218260451', '_score': 14.8171215, '_source': {'numeroProcesso': '15000156520218260451', 'classe': {'codigo': 279, 'nome': 'Inquérito Policial'}, 'sistema': {'codigo': 3, 'nome': 'SAJ'}, 'formato': {'codigo': 1, 'nome': 'Eletrônico'}, 'tribunal': 'TJSP', 'dataHoraUltimaAtualizacao': '2023-07-20T02:27:24.007Z', 'grau': 'G1', '@timestamp': '2023-09-11T02:24:52.717Z', 'dataAjuizamento': '2021-01-05T10:29:06.000Z', 'movimentos': [{'complementosTabelados': [{'codigo': 2, 'valor': 2, 'nome': 'sorteio', 'descricao': 'tipo_de_distribuicao_redistribuicao'}], 'codigo': 26, 'nome': 'Distribuição', 'dataHora': '2021-01-05T10:29:26.000Z'}, {'complementosTabelados': [{'codigo': 19, 'valor': 57, 'nome': 'Petição (outras)', 'descricao

In [3]:
# Descobrindo código e órgão julgador 
codigo = dados_dict['hits']['hits'][0]['_source']['orgaoJulgador']['codigo']
orgao_julgador = dados_dict['hits']['hits'][0]['_source']['orgaoJulgador']['nome']

print (f'Código: {codigo} - Órgão julgador: {orgao_julgador}')

Código: 10067 - Órgão julgador: 03 CRIMINAL DE PIRACICABA


In [4]:
url = "https://api-publica.datajud.cnj.jus.br/api_publica_tjsp/_search"
api_key = 'APIKey cDZHYzlZa0JadVREZDJCendQbXY6SkJlTzNjLV9TRENyQk1RdnFKZGRQdw==' # Chave pública
orgao_julgador = 10071 # Exemplos: 10066 10065 10067 59205 10071

payload = json.dumps({
  "size": 10000,
  "query": {
    "match": {"orgaoJulgador.codigo": orgao_julgador
    #"match": {"orgaoJulgador.codigoMunicipioIBGE": 3538709             
    }
  },
  "sort": [{"dataAjuizamento": {"order": "desc"}}] # ou asc
})

headers = {
  'Authorization': api_key,
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)
dados_dict = json.loads(response.text)

In [5]:
dados_dict['hits']['hits'][0]

{'_index': 'api_publica_tjsp',
 '_type': '_doc',
 '_id': 'TJSP_386_G1_10071_00071373820238260451',
 '_score': None,
 '_source': {'classe': {'codigo': 386, 'nome': 'Execução da Pena'},
  'numeroProcesso': '00071373820238260451',
  'sistema': {'codigo': 3, 'nome': 'SAJ'},
  'formato': {'codigo': 1, 'nome': 'Eletrônico'},
  'tribunal': 'TJSP',
  'dataHoraUltimaAtualizacao': '2023-08-30T08:23:41.169Z',
  'grau': 'G1',
  '@timestamp': '2023-10-17T02:04:12.283Z',
  'dataAjuizamento': '2023-07-31T16:13:40.000Z',
  'movimentos': [{'complementosTabelados': [{'codigo': 2,
      'valor': 1,
      'nome': 'competência exclusiva',
      'descricao': 'tipo_de_distribuicao_redistribuicao'}],
    'codigo': 26,
    'nome': 'Distribuição',
    'dataHora': '2023-07-31T16:20:52.000Z'},
   {'codigo': 11383,
    'nome': 'Ato ordinatório',
    'dataHora': '2023-08-03T09:35:19.000Z'},
   {'complementosTabelados': [{'codigo': 4,
      'valor': 79,
      'nome': 'Ofício',
      'descricao': 'tipo_de_documento'}

In [6]:
processos = []

for processo in dados_dict['hits']['hits']:
  numero_processo = processo['_source']['numeroProcesso']
  grau = processo['_source']['grau']
  classe = processo['_source']['classe']['nome']
  assuntos = processo['_source']['assuntos'] # Pode ter mais de um
  data_ajuizamento = processo['_source']['dataAjuizamento']
  ultima_atualizacao = processo['_source']['dataHoraUltimaAtualizacao']
  formato = processo['_source']['formato']['nome']
  codigo = processo['_source']['orgaoJulgador']['codigo']
  orgao_julgador = processo['_source']['orgaoJulgador']['nome']
  municipio = processo['_source']['orgaoJulgador']['codigoMunicipioIBGE']
  sort = processo['sort'][0]
  try:
    movimentos = processo['_source']['movimentos']
  except:
    movimentos = []

  processos.append([numero_processo, classe, data_ajuizamento, ultima_atualizacao, formato, \
                    codigo, orgao_julgador, municipio, grau, assuntos, movimentos, sort])

df = pd.DataFrame(processos, columns=['numero_processo', 'classe', 'data_ajuizamento', 'ultima_atualizacao', \
                      'formato', 'codigo', 'orgao_julgador', 'municipio', 'grau', 'assuntos', 'movimentos', 'sort'])   

df.sample(5)               

,numero_processo,classe,data_ajuizamento,ultima_atualizacao,formato,codigo,orgao_julgador,municipio,grau,assuntos,movimentos,sort
9334,00021506120208260451,Execução da Pena,2020-02-20T14:05:00.000Z,2023-07-20T02:10:20.774Z,Eletrônico,10071,JURI/EXECUCOES DE PIRACICABA,3538709,G1,"[{'codigo': 7785, 'nome': 'Prestação Pecuniári...","[{'complementosTabelados': [{'codigo': 2, 'val...",1582207500000
8867,70003199420208260114,Execução da Pena,2020-05-22T00:00:00.000Z,2023-08-30T05:22:43.114Z,Eletrônico,10071,JURI/EXECUCOES DE PIRACICABA,3538709,G1,"[{'codigo': 7791, 'nome': 'Pena Privativa de L...","[{'codigo': 11383, 'nome': 'Ato ordinatório', ...",1590105600000
8508,00003792620198260502,Remição,2020-07-17T13:37:20.000Z,2023-07-20T03:29:05.415Z,Eletrônico,10071,JURI/EXECUCOES DE PIRACICABA,3538709,G1,"[{'codigo': 10635, 'nome': 'Progressão de Regi...","[{'codigo': 981, 'nome': 'Recebimento', 'dataH...",1594993040000
1759,10181781920228260451,Execução de Medidas Alternativas no Juízo Comum,2022-09-21T14:05:06.000Z,2023-08-30T08:16:25.867Z,Eletrônico,10071,JURI/EXECUCOES DE PIRACICABA,3538709,G1,"[{'codigo': 12730, 'nome': 'Acordo de Não Pers...","[{'complementosTabelados': [{'codigo': 2, 'val...",1663769106000
8374,00069928420208260451,Execução da Pena,2020-08-07T10:16:17.000Z,2023-07-20T10:45:35.178Z,Eletrônico,10071,JURI/EXECUCOES DE PIRACICABA,3538709,G1,"[{'codigo': 7788, 'nome': 'Interdição Temporár...","[{'complementosTabelados': [{'codigo': 2, 'val...",1596795377000


In [7]:
def converte_data(data_str):
    return pd.to_datetime(data_str).tz_convert('America/Sao_Paulo')


def gera_lista_assuntos(assuntos_do_df):
    lst_assuntos=[]
    for assunto in assuntos_do_df:
        try:
            lst_assuntos.append(assunto.get('nome'))
        except:
            lst_assuntos.append('')

    return lst_assuntos


def gera_lista_movimentos(movimentos):
    lst_movimentos_final =[]
    for movimento in movimentos:
        codigo = movimento.get('codigo')
        nome = movimento.get('nome')
        data_hora = movimento.get('dataHora')
        if data_hora:
            data_hora = converte_data(data_hora)
        lst_movimentos_final.append([codigo, nome, data_hora])
    return lst_movimentos_final

In [8]:
df['assuntos'] = df['assuntos'].apply(gera_lista_assuntos)
df['movimentos'] = df['movimentos'].apply(gera_lista_movimentos)
df['data_ajuizamento'] = df['data_ajuizamento'].apply(converte_data)
df['ultima_atualizacao'] = df['ultima_atualizacao'].apply(converte_data)
df['movimentos']= df['movimentos'].apply(lambda x: sorted(x, key=lambda tup: tup[2], reverse=False))
df.sample(5)

,numero_processo,classe,data_ajuizamento,ultima_atualizacao,formato,codigo,orgao_julgador,municipio,grau,assuntos,movimentos,sort
8268,00072899120208260451,Execução da Pena,2020-08-17 12:29:45-03:00,2023-07-19 23:23:49.897000-03:00,Eletrônico,10071,JURI/EXECUCOES DE PIRACICABA,3538709,G1,[Prestação Pecuniária],"[[26, Distribuição, 2020-08-17 12:36:18-03:00]...",1597678185000
6901,00013192020218260502,Execução da Pena,2021-02-05 07:59:10-03:00,2023-07-20 00:31:10.463000-03:00,Eletrônico,10071,JURI/EXECUCOES DE PIRACICABA,3538709,G1,[Progressão de Regime],"[[26, Distribuição, 2021-02-05 07:59:10-03:00]...",1612522750000
4561,70000837220218260320,Execução da Pena,2021-11-02 21:00:00-03:00,2023-07-20 07:44:23.725000-03:00,Eletrônico,10071,JURI/EXECUCOES DE PIRACICABA,3538709,G1,[Pena Privativa de Liberdade],"[[11383, Ato ordinatório, 2021-11-02 21:00:00-...",1635897600000
3120,10086038420228260451,Execução de Pena de Multa,2022-05-13 15:50:08-03:00,2023-07-19 23:39:35.598000-03:00,Eletrônico,10071,JURI/EXECUCOES DE PIRACICABA,3538709,G1,[Pena de Multa],"[[26, Distribuição, 2022-05-16 12:12:46-03:00]...",1652467808000
6115,10087998820218260451,Execução de Pena de Multa,2021-05-17 11:52:46-03:00,2023-08-30 05:12:57.877000-03:00,Eletrônico,10071,JURI/EXECUCOES DE PIRACICABA,3538709,G1,[Pena de Multa],"[[26, Distribuição, 2021-05-17 13:28:06-03:00]...",1621263166000


In [9]:
# Gravação dos arquivos

df.to_csv('processos.csv', sep=',', header=True, index=False)

with open('resultado.txt', 'w') as file:
    for index, row in df.iterrows():
        file.write(f"Autos n. {row['numero_processo']}, Classe: {row['classe']}\n") 
        file.write(f"Ajuizamento: {row['data_ajuizamento']}\n")
        file.write(f"Assuntos: {row['assuntos']}\n\n")
        for movimento in row['movimentos']:  
            file.write(f"{str(movimento[2])} | Cód: {str(movimento[0])} | Mov.: {movimento[1]} \n")
        file.write('\n-----------------------------------------------------------------------------\n\n')